In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
    test_loss /= len(test_loader.dataset)
 
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
def main():
 
    device =  "cpu"
 
    train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
    for epoch in range(0, 10):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
 
 
    torch.save(model.state_dict(),"output/model.pt")
 
        
if __name__ == '__main__':
    main()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.306687
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.660943
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.411799
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.270197
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.225897
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.139140
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.170813
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.269650
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.191982
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.143769

Test set: Average loss: 0.0892, Accuracy: 9737/10000 (97%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.162235
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.029896
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.130882
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.094761
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.053398
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.043981
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.080040
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.014838
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.123765
T

In [ ]:
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import torch.nn as nn
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
trained_model = Net()
trained_model.load_state_dict(torch.load('output/model.pt'))
dummy_input = Variable(torch.randn(1, 1, 28, 28)) 
torch.onnx.export(trained_model, dummy_input, "output/model.onnx") 